In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import csv
import math
import json
import ast
import cv2
import warnings
import seaborn as sns
import random

np.random.seed(42)
random.seed(42)

# Ignore warnings
warnings.filterwarnings('ignore')

## Dataset 2: one minute - one data point

In [22]:
aggre={
'IndTrig_L' : 'sum',
'IndTrig_R' : 'sum',
'HandTrig_L' : 'sum',
'HandTrig_R' : 'sum',
'Thumb_L_x' : 'sum',
'Thumb_R_x' : 'sum',
'Thumb_L_y' : 'sum',
'Thumb_R_y' : 'sum',
'le_roll' : 'median_change',
'le_pitch' : 'median_change',
'le_yaw' : 'median_change',
'LE_speed' : 'median' ,
'h_roll' : 'median_change',
'h_pitch' : 'median_change',
'h_yaw' : 'median_change',
'Head_Speed' : 'median' , 
'Head_Velocity_Change' : 'median' ,
'Head_AngVel_Change' : 'median',
'c_roll' : 'median_change',
'c_pitch' : 'median_change',
'c_yaw' : 'median_change',
'c_speed' : 'median',
'text_presence' : 'sum',
'change in brightness' : 'sum',
'diff_HE_yaw' : 'median',
'diff_HE_roll' : 'median',
'diff_HE_pitch' : 'median',
'c_acceleration1' : 'median',
'c_acceleration2' : 'median',
'c_acceleration3' : 'median',
'c_velocity1' : 'median',
'c_velocity2' : 'median',
'c_velocity3' : 'median'}

# Ignore warnings
warnings.filterwarnings('ignore')

def expand_cols(df, column_name):
    df[column_name]= df[column_name].apply(lambda x: ast.literal_eval(x))
    data_expanded = pd.DataFrame(df[column_name].tolist(), index=df.index)
    data_expanded.columns = [column_name+'1', column_name+'2', column_name+'3']
    df = df.drop(columns=[column_name]).join(data_expanded)
    return df

def change_diff(df, column_list):
    for column in column_list:
        df[column+'_change']=df[column]-df[column].shift(1)
    return df
    
def aggregate_df(df,aggre):
    df1=expand_cols(df,'c_acceleration')
    df2=expand_cols(df1,'c_velocity')
    change=[]
    for i in aggre.keys():
        if aggre[i]=='median_change':
            change.append(i)
            aggre.pop(i)
            aggre[i+'_change']='median'
    df=change_diff(df2,change)
    aggregated_df = df.agg(aggre)
    return pd.DataFrame(aggregated_df).transpose()


with open('/home/sharedFolder/sessions.json', 'r') as file:
    sessions = json.load(file)

dfs=[]
count=0
session_saving={}

In [23]:
sessions={'Earth_Gym': ['5_2_Earth_Gym',
  '10_2_Earth_Gym']}

In [14]:
def create_frames_1(df_m, aggre,j):
    df=df_m.copy()
    index_list= df[df['MS_rating'].notna()].index.tolist()
    msr=df['MS_rating'].dropna().tolist()
    # print(msr)
    # index_list.insert(0, 0)
    df3=df.iloc[0:1]
    # print(df2)
    # dfs1=df3.copy()
    count1=0
    for i in range(len(index_list) - 1):
        if count1==0:
            start_idx=0
            end_idx=1
            df2=df.iloc[start_idx:end_idx]
            aggre1=aggre.copy()
            df2=aggregate_df(df2.copy(), aggre1)
            # msr=df2['MS_rating'].dropna().max()
            dfs1=df2.copy()
        start_idx = index_list[i]
        end_idx = index_list[i + 1]  
        # df2=df.copy()
        if count==1:
            start_idx+=1
        # print(start_idx, end_idx)
        df2=df.iloc[start_idx:end_idx]
        # print(df2.columns)
        aggre1=aggre.copy()
        df2=aggregate_df(df2.copy(), aggre1)
        # else:
        # print(df2)
        df_c1=[dfs1,df2]
        # return dfs1
        dfs1 = pd.concat(df_c1, ignore_index=True)
        
        # return dfs1
        count1+=1
        # print(count1)
    dfs1['msr']=msr
    dfs1['session']=j
    return dfs1
        

In [25]:
for i in sessions.keys():
    # logger.info(f'***************{i}********************')
    for j in sessions[i]:
        # logger.info(f'*******{j}********')
        path=os.path.join('/data/VR_NET/folders/', i, j, 'data_file_2.csv')
        print(path)
        df1=pd.read_csv(path)
        df=df1.copy()
        # try:
        df=create_frames_1(df, aggre, j)
        # print(df)
        # cleaned_dfs.append(df)
        if count==0:
            dfs=df.copy()
        else:
            df_c=[dfs,df]
            dfs = pd.concat(df_c)
        session_saving[count]=path
        count+=1
            # logger.info('*******successfully saved********')
        # except:
        #     print('failed')
            # logger.info('*******failed********')

/data/VR_NET/folders/Earth_Gym/5_2_Earth_Gym/data_file_2.csv
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
/data/VR_NET/folders/Earth_Gym/10_2_Earth_Gym/data_file_2.csv
[1.0, 1.0, 2.0, 1.0, 3.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0]


# Dataset 3: two_minutes - one data point

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import csv
import math
import json
import ast
import cv2
import warnings
import seaborn as sns
import random
from logger_config import setup_logger

# Get logger
logger = setup_logger()


np.random.seed(42)
random.seed(42)

# Ignore warnings
warnings.filterwarnings('ignore')


aggre={
'IndTrig_L' : 'sum',
'IndTrig_R' : 'sum',
'HandTrig_L' : 'sum',
'HandTrig_R' : 'sum',
'Thumb_L_x' : 'sum',
'Thumb_R_x' : 'sum',
'Thumb_L_y' : 'sum',
'Thumb_R_y' : 'sum',
'le_roll' : 'median_change',
'le_pitch' : 'median_change',
'le_yaw' : 'median_change',
'LE_speed' : 'median' ,
'h_roll' : 'median_change',
'h_pitch' : 'median_change',
'h_yaw' : 'median_change',
'Head_Speed' : 'median' , 
'Head_Velocity_Change' : 'median' ,
'Head_AngVel_Change' : 'median',
'c_roll' : 'median_change',
'c_pitch' : 'median_change',
'c_yaw' : 'median_change',
'c_speed' : 'median',
'text_presence' : 'sum',
'change in brightness' : 'sum',
'diff_HE_yaw' : 'median',
'diff_HE_roll' : 'median',
'diff_HE_pitch' : 'median',
'c_acceleration1' : 'median',
'c_acceleration2' : 'median',
'c_acceleration3' : 'median',
'c_velocity1' : 'median',
'c_velocity2' : 'median',
'c_velocity3' : 'median'}

# Ignore warnings
warnings.filterwarnings('ignore')

def expand_cols(df, column_name):
    df[column_name]= df[column_name].apply(lambda x: ast.literal_eval(x))
    data_expanded = pd.DataFrame(df[column_name].tolist(), index=df.index)
    data_expanded.columns = [column_name+'1', column_name+'2', column_name+'3']
    df = df.drop(columns=[column_name]).join(data_expanded)
    return df

def change_diff(df, column_list):
    for column in column_list:
        df[column+'_change']=df[column]-df[column].shift(1)
    return df
    
def aggregate_df(df,aggre):
    df1=expand_cols(df,'c_acceleration')
    df2=expand_cols(df1,'c_velocity')
    change=[]
    for i in aggre.keys():
        if aggre[i]=='median_change':
            change.append(i)
            aggre.pop(i)
            aggre[i+'_change']='median'
    df=change_diff(df2,change)
    aggregated_df = df.agg(aggre)
    return pd.DataFrame(aggregated_df).transpose()


        


with open('/home/sharedFolder/sessions.json', 'r') as file:
    sessions = json.load(file)

dfs=[]
count=0

sessions={'Earth_Gym': ['5_2_Earth_Gym',
  '10_2_Earth_Gym']}

In [11]:
def create_frames_1(df_m, aggre,j):
    df=df_m.copy()
    index_list= df[df['MS_rating'].notna()].index.tolist()
    msr=df['MS_rating'].dropna().tolist()
    df3=df.iloc[0:1]
    count1=0
    for i in range(len(index_list) - 2):
        # if count1==0:
        #     start_idx=0
        #     end_idx=1
        #     df2=df.iloc[start_idx:end_idx]
        #     aggre1=aggre.copy()
        #     df2=aggregate_df(df2.copy(), aggre1)
        #     dfs1=df2.copy()
        start_idx = index_list[i]
        end_idx = index_list[i + 2]  
        if count==1:
            start_idx+=1
        df2=df.iloc[start_idx:end_idx]
        aggre1=aggre.copy()
        df2=aggregate_df(df2.copy(), aggre1)
        if count1 ==0:
            dfs1=df2.copy()
        else:
            df_c1=[dfs1,df2]
            dfs1 = pd.concat(df_c1, ignore_index=True)
        count1+=1
        print(start_idx, end_idx)
    print(len(dfs1))
    dfs1['msr']=msr[2:]
    dfs1['msr_1']=msr[1:-1]
    dfs1['msr_2']=msr[:-2]
    dfs1['session']=j
    return dfs1

In [13]:
for i in sessions.keys():
    # logger.info(f'***************{i}********************')
    for j in sessions[i]:
        # logger.info(f'*******{j}********')
        path=os.path.join('/data/VR_NET/folders/', i, j, 'data_file_2.csv')
        print(path)
        df1=pd.read_csv(path)
        df=df1.copy()
        # try:
        df=create_frames_1(df, aggre, j)
        # print(df)
        # cleaned_dfs.append(df)
        if count==0:
            dfs=df.copy()
        else:
            df_c=[dfs,df]
            dfs = pd.concat(df_c)
        # session_saving[count]=path
        count+=1
            # logger.info('*******successfully saved********')
        # except:
        #     print('failed')
            # logger.info('*******failed********')

/data/VR_NET/folders/Earth_Gym/5_2_Earth_Gym/data_file_2.csv
0 588
222 987
588 1337
987 1788
1337 2157
1788 2567
2157 2986
2567 3384
2986 3820
3384 4279
3820 4618
11
/data/VR_NET/folders/Earth_Gym/10_2_Earth_Gym/data_file_2.csv
2 900
430 1306
901 1698
1307 2260
1699 2565
2261 2985
2566 3528
2986 3878
3529 4239
3879 4741
4240 5119
11


In [14]:
dfs

,IndTrig_L,IndTrig_R,HandTrig_L,HandTrig_R,Thumb_L_x,Thumb_R_x,Thumb_L_y,Thumb_R_y,LE_speed,Head_Speed,...,h_roll_change,h_pitch_change,h_yaw_change,c_roll_change,c_pitch_change,c_yaw_change,msr,msr_1,msr_2,session
0,8.000000,8.168498,51.239068,19.478632,17.081848,4.663414,10.019126,6.656926,0.042908,0.049192,...,-0.000797,-0.001815,0.000335,-0.000161,0.000195,0.000000,1.0,1.0,1.0,5_2_Earth_Gym
1,23.629548,151.273748,96.206344,124.476189,42.732780,8.357490,8.872966,1.773924,0.040971,0.040261,...,-0.002170,-0.000581,0.000969,-0.000453,-0.000169,0.000725,1.0,1.0,1.0,5_2_Earth_Gym
2,21.594627,369.824175,66.768496,129.971179,28.309321,4.011695,-3.275341,0.466922,0.048111,0.046984,...,-0.001974,-0.000504,0.001697,-0.000357,0.000218,0.000934,1.0,1.0,1.0,5_2_Earth_Gym
3,99.069597,365.407570,121.239313,62.329412,19.922779,1.139057,2.422207,8.646779,0.081222,0.080344,...,-0.001360,-0.000175,-0.000527,0.002059,0.000284,0.001357,1.0,1.0,1.0,5_2_Earth_Gym
4,107.317948,332.018803,136.711833,76.742109,21.314843,-2.757746,4.034744,1.630032,0.078037,0.076945,...,-0.002277,0.000328,-0.000642,-0.001882,0.000227,0.004574,1.0,1.0,1.0,5_2_Earth_Gym
5,27.312820,432.603174,109.687655,130.765553,4.428476,-1.519324,-0.646609,-0.818831,0.079755,0.079859,...,0.000749,-0.000650,0.000325,-0.007749,0.000120,0.008105,1.0,1.0,1.0,5_2_Earth_Gym
6,39.091331,498.377044,141.991200,296.091804,9.025706,1.005332,-5.483141,0.253941,0.066904,0.068931,...,-0.000538,-0.000077,-0.000019,-0.000793,-0.000209,0.001168,1.0,1.0,1.0,5_2_Earth_Gym
7,67.781685,483.981438,98.408055,355.061285,14.073743,-0.211967,-8.769522,1.705341,0.039280,0.039796,...,-0.000962,0.000049,-0.000481,0.000781,0.000200,0.001077,1.0,1.0,1.0,5_2_Earth_Gym
8,56.564591,494.745297,63.386811,200.623194,16.592093,-0.643063,0.166827,0.574693,0.049953,0.049731,...,-0.001190,0.000752,-0.000425,0.002391,0.000287,0.001111,1.0,1.0,1.0,5_2_Earth_Gym
9,38.465445,423.032722,18.897189,83.182412,28.430035,3.506885,9.264001,-0.580261,0.054485,0.054047,...,-0.003013,0.000947,-0.000161,0.000925,0.000345,0.001596,1.0,1.0,1.0,5_2_Earth_Gym
